In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# https://huggingface.co/facebook/nllb-200-distilled-600M

model_checkpoint = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt", src_lang='kor_Hang', tgt_lang='eng_Latn')
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [3]:
# dataset
from datasets import load_dataset
dataset = load_dataset('json', data_files='./일상생활및구어체_영한_train_set.json', field='data')

split_datasets = dataset["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

# declare preprocessing code 

max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = examples['ko']
    targets = examples['en']

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # 타겟을 위한 토크나이저 셋업
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs            



tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Found cached dataset json (/home/juhwan/.cache/huggingface/datasets/json/default-23e8ad3d1a915c98/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)
100%|██████████| 1/1 [00:00<00:00, 18.92it/s]
Loading cached split indices for dataset at /home/juhwan/.cache/huggingface/datasets/json/default-23e8ad3d1a915c98/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-cfd229826466e59a.arrow and /home/juhwan/.cache/huggingface/datasets/json/default-23e8ad3d1a915c98/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-61c3fa9b299534e5.arrow
Loading cached processed dataset at /home/juhwan/.cache/huggingface/datasets/json/default-23e8ad3d1a915c98/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-8eb4d536ff23d0d0.arrow
Loading cached processed dataset at /home/juhwan/.cache/huggingface/datasets/json/default-23e8ad3d1a915c98/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-b74

In [4]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [5]:
from datasets import load_metric
metric = load_metric("sacrebleu")

/tmp/ipykernel_15533/3063495125.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [6]:
# # 코드 자세히봐야해.. 왜 -100을 제거하고 그런거 알아봐야한다.
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [7]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"nllb-finetuned-aihub-ko-to-en",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    eval_accumulation_steps=1,
)

In [8]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [14]:
tokenized_datasets['validation']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 120031
})

In [15]:
prediction = trainer.predict(tokenized_datasets["validation"])

/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


KeyboardInterrupt: 

In [15]:
# 학습하기전 초기모델 점수
trainer.evaluate(max_length=max_target_length)

You're using a NllbTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.4023785591125488,
 'eval_bleu': 0.0162,
 'eval_gen_len': 13.5995,
 'eval_runtime': 1549.4907,
 'eval_samples_per_second': 77.465,
 'eval_steps_per_second': 0.303}

In [16]:
trainer.train()

/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate(max_length=max_target_length)